In [3]:

!pip install --upgrade ultralytics
!pip install mediapipe


  Using cached ultralytics-8.3.135-py3-none-any.whl.metadata (37 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached torch-2.7.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached torchvision-0.22.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached charset_normalizer-3.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cac

In [ ]:
# for_YOLO is directory name with YOLOv11 format for training
#!unzip /content/for_YOLO.zip

Archive:  /content/for_YOLO.zip
   creating: for_YOLO/
  inflating: __MACOSX/._for_YOLO     
  inflating: for_YOLO/.DS_Store      
  inflating: __MACOSX/for_YOLO/._.DS_Store  
  inflating: for_YOLO/info.txt       
  inflating: __MACOSX/for_YOLO/._info.txt  
   creating: for_YOLO/train/
  inflating: __MACOSX/for_YOLO/._train  
   creating: for_YOLO/val/
  inflating: __MACOSX/for_YOLO/._val  
   creating: for_YOLO/train/healthy/
  inflating: __MACOSX/for_YOLO/train/._healthy  
   creating: for_YOLO/train/unhealthy/
  inflating: __MACOSX/for_YOLO/train/._unhealthy  
   creating: for_YOLO/val/healthy/
  inflating: __MACOSX/for_YOLO/val/._healthy  
   creating: for_YOLO/val/unhealthy/
  inflating: __MACOSX/for_YOLO/val/._unhealthy  
  inflating: for_YOLO/train/healthy/_1580971992669_jpg.rf.b44f5b31a6ce6f449462b637ca8eb0bf.jpg  
  inflating: __MACOSX/for_YOLO/train/healthy/.__1580971992669_jpg.rf.b44f5b31a6ce6f449462b637ca8eb0bf.jpg  
  inflating: for_YOLO/train/healthy/_1582960728025_jpg.rf

In [4]:

from ultralytics import YOLO

# loading classification model
# model = YOLO('yolo11n-cls.pt')

# train the model
# results = model.train(data='../../../../data/processed/for_YOLO', epochs=400)


In [ ]:


from IPython.display import Image



Image("runs/classify/train/results.png", width=600)



In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os

FACE_LANDMARKS = [10, 109, 67, 103, 54, 21, 162, 127, 227, 137, 177, 215, 138, 135, 136, 169, 
                  150, 149, 176, 148, 152, 377, 400, 378, 379, 365, 364, 397, 435, 401, 
                  366, 447, 366, 389, 251, 284, 332, 297, 338]

# Инициализация MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

def process_image(image_path, output_path):
    """Обрабатывает одно изображение, подготавливая его к inference и сохраняет результат"""
    image = cv2.imread(image_path)
    if image is None:
        return False
    
    # Конвертируем в RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Обработка лица
    results = face_mesh.process(image_rgb)
    
    if results.multi_face_landmarks:
        h, w = image.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)
        
        for face_landmarks in results.multi_face_landmarks:
            points = []
            for idx in FACE_LANDMARKS:
                if idx < len(face_landmarks.landmark):
                    lm = face_landmarks.landmark[idx]
                    x, y = int(lm.x * w), int(lm.y * h)
                    points.append((x, y))
            
            if len(points) > 2:
                points_array = np.array(points, dtype=np.int32)
                cv2.fillPoly(mask, [points_array], 255)
                
                # Находим ограничивающий прямоугольник
                x, y, w_rect, h_rect = cv2.boundingRect(points_array)
                
                # Вырезаем область лица с небольшим запасом
                padding = 20
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(w, x + w_rect + padding)
                y2 = min(h, y + h_rect + padding)
                
                cropped = image[y1:y2, x1:x2]
                
                # Сохраняем обрезанное изображение
                output_filename = os.path.join(output_path, os.path.basename(image_path))
                cv2.imwrite(output_filename, cropped)
                return True
    
    return False

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1747374008.347591 3279702 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747374008.367299 3279702 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:


model = YOLO('classify/train/weights/best.pt')

process_image("a294bba19001ec335a702476a3430594.jpg", "./")

image = cv2.imread("a294bba19001ec335a702476a3430594.jpg")

results = model(image) #, save=True, conf=0.5)
results[0].show()

W0000 00:00:1747374031.638458 3279703 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
results = model("/content/healthy.0.jpg", save=True, conf=0.5)
results[0].show()

In [ ]:
results